In [1]:
import pandas as pd
import numpy as np
import time
import psutil
import subprocess
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

In [2]:
data= pd.read_csv(r"C:\Users\hsynt\Downloads\Selcuk Abi\Selcuk Abi\VeriSeti2.csv")

In [3]:
# GPU Kullanımını Ölçmek İçin Fonksiyon
def get_gpu_usage():
    try:
        result = subprocess.check_output(["nvidia-smi", "--query-gpu=utilization.gpu", "--format=csv,noheader,nounits"])
        gpu_usage = float(result.decode("utf-8").strip())
    except:
        gpu_usage = 0.0  # GPU yoksa veya erişilemezse 0 olarak döndür
    return gpu_usage

# CPU ve Bellek Kullanımını Ölçmek İçin Fonksiyon
def get_cpu_memory_usage():
    cpu_usage = psutil.cpu_percent(interval=1)
    memory_usage = psutil.virtual_memory().used / (1024 ** 3)  # GB cinsinden
    return cpu_usage, memory_usage

# Özellik ve Hedef Ayrımı
X = data.drop(columns=['Target_10', 'adjclose'])
y = data['Target_10']

# Zaman Serisi Formatına Getirme
def create_sequences(data, target, seq_length):
    sequences, targets = [], []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        targets.append(target[i + seq_length])
    return np.array(sequences), np.array(targets)

SEQ_LENGTH = 10
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_seq, y_seq = create_sequences(X_scaled, y.values, SEQ_LENGTH)

# PCA ve LDA Veri Hazırlığı
X_flat = X_seq.reshape(X_seq.shape[0], -1)  # Düzleştirilmiş veri

pca = PCA(n_components=10)
X_pca = pca.fit_transform(X_flat)

lda = LDA(n_components=1)
X_lda = lda.fit_transform(X_flat, y_seq)

# LSTM Modeli Tanımlama (Özellik Çıkartma için)
def build_lstm_feature_extractor(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(64, activation='relu', return_sequences=False),
        Dropout(0.2),
        Dense(32, activation='relu')  # Ara katman: Özellik Çıkarıcı
    ])
    return model

# Performans Hesaplama
results = []

for reduction, X_data in zip(
    ['No PCA/LDA', 'PCA', 'LDA'],
    [X_seq, X_pca, X_lda]
):
    if reduction == 'No PCA/LDA':
        X_data = X_data.reshape(X_data.shape[0], SEQ_LENGTH, -1)

        # LSTM Modeli ile Özellik Çıkartma
        lstm_model = build_lstm_feature_extractor(X_data.shape[1:])
        start_time = time.time()
        X_features = lstm_model.predict(X_data)
        lstm_time = time.time() - start_time  # LSTM Özellik Çıkartma Süresi

    else:
        # PCA ve LDA için düzleştirilmiş veriler kullanılır
        X_features = X_data
        lstm_time = 0  # LSTM Eğitim Süresi Yok

    # Random Forest Modeli
    rf_model = RandomForestClassifier(n_estimators=200, random_state=42)

    # Modeli Eğit ve Kullanım Ölçümleri
    start_time = time.time()
    rf_model.fit(X_features, y_seq)
    rf_time = time.time() - start_time

    # Kullanım Ölçümleri
    gpu_usage = get_gpu_usage()
    cpu_usage, memory_usage = get_cpu_memory_usage()

    results.append({
        'Reduction': reduction,
        'Model': 'LSTM + Random Forest',
        'CPU Usage (%)': cpu_usage,
        'GPU Usage (%)': gpu_usage,
        'Memory Usage (GB)': memory_usage
    })

# Performans Sonuçlarını Görselleştirme
results_df = pd.DataFrame(results)

# CPU, GPU ve Bellek Kullanımı Tablosu
print("CPU, GPU ve Bellek Kullanımı Tablosu:")
print(results_df)

# Sonuçları Kaydetme
results_df.to_csv("lstm_rf_resource_usage.csv", index=False)
print("\nSonuçlar 'lstm_rf_resource_usage.csv' dosyasına kaydedildi.")

1562/1562 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
CPU, GPU ve Bellek Kullanımı Tablosu:
    Reduction                 Model  CPU Usage (%)  GPU Usage (%)  \
0  No PCA/LDA  LSTM + Random Forest            0.3            0.0   
1         PCA  LSTM + Random Forest            0.4            0.0   
2         LDA  LSTM + Random Forest            1.8            0.0   

   Memory Usage (GB)  
0          16.421581  
1          15.789772  
2          15.792950  

Sonuçlar 'lstm_rf_resource_usage.csv' dosyasına kaydedildi.
